# Log in to huggingface

In [ ]:
from huggingface_hub import login
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

if HF_TOKEN:
    login(HF_TOKEN)
    print("Successfully logged in to Hugging Face!")
else:
    print("Token is not set. Please save the token first.")

Successfully logged in to Hugging Face!


# Installing prereqs

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 41.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q peft transformers datasets

In [ ]:
!pip install llama-stack -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.9/369.9 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.9/444.9 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      

In [ ]:
!pip install keras-adamw

# Imports

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import get_scheduler
from tqdm import tqdm
import time
import os

In [ ]:
import os
import time
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    get_scheduler,
)
from peft import LoraConfig, get_peft_model, TaskType
from torch.optim import AdamW

# Load Model and params

In [ ]:
torch.cuda.is_available()

True

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B"
save_dir = "/content/lora_checkpoints"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
!nvidia-smi
gpu_name = torch.cuda.get_device_name(0)
print(f"Using GPU: {gpu_name}")

Sat Sep 20 22:11:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             44W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
if "A100" in gpu_name:
    device_map = {"": 0}  # put everything on GPU
else:
    device_map = "balanced_low_0"  # spread layers GPU+CPU

In [ ]:
# Model itself
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_use_double_quant=True,
    bnb_8bit_quant_type="nf4",
    llm_int8_enable_fp32_cpu_offload=True
)
print(f"Loading model with device_map={device_map}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map=device_map,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # needed for batching

Loading model with device_map={'': 0}...


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [ ]:
# freeze base model
for param in model.parameters():
    param.requires_grad = False

# Lora config


In [ ]:
# -----------------------
# LoRA config
# -----------------------
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

print("Applying LoRA adapters...")
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Applying LoRA adapters...
trainable params: 3,407,872 || all params: 8,033,669,120 || trainable%: 0.0424


# Dataset

#V2

In [ ]:
torch.save(model.state_dict(), 'base_initial_model.pt')

In [ ]:
batch = next(iter(train_loader))
print(batch.keys())
print(batch["input_ids"].shape)


KeysView({'input_ids': tensor([[128000,  20421,    220,   3174,    198,  19285],
        [128000,  19285,    198,  19285, 128001, 128001]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0]]), 'labels': None})
torch.Size([2, 6])


In [ ]:
csv_path = "/content/new_all_podcasts.csv"
output_file = "file_clean.csv"

good_rows = []

with open(csv_path, "r", encoding="utf-8") as f:
    header = f.readline()
    good_rows.append(header)
    for line in f:
        try:
            if len(line.strip().split(",")) < 2:
                continue
            good_rows.append(line.replace('"', ''))
        except Exception:
            continue

with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(good_rows)

df = pd.read_csv(output_file, on_bad_lines="skip")
print("Rows after cleaning:", len(df))
print(df.head())

# -----------------------
# 2️⃣ Tokenizer + chunking
# -----------------------
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.pad_token = tokenizer.eos_token  # causal LM doesn’t have a pad token

max_tokens = 256
stride = 50

def chunk_text(text, max_tokens=max_tokens, stride=stride):
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens - stride):
        chunk_tokens = tokens[i:i + max_tokens]
        if not chunk_tokens:
            continue
        chunks.append(chunk_tokens)
        if i + max_tokens >= len(tokens):
            break
    return chunks

rows = []
for idx, row in df.iterrows():
    title = str(row["Title"])
    transcript = str(row["Transcript"])
    token_chunks = chunk_text(transcript)
    for chunk_tokens in token_chunks:
        # combine title + chunk tokens as text
        text = f"{title}\n{tokenizer.decode(chunk_tokens, skip_special_tokens=True)}"
        rows.append({"text": text})

# -----------------------
# 3️⃣ Hugging Face Dataset
# -----------------------
dataset = Dataset.from_list(rows)

def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=max_tokens
    )

dataset = dataset.map(tokenize_fn, batched=True)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
print("Dataset ready! Number of examples:", len(dataset))

# -----------------------
# 4️⃣ DataLoader for causal LM
# -----------------------
batch_size = 2 if torch.cuda.is_available() else 1
collate_fn = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # causal LM
)

train_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

Rows after cleaning: 754
                                                                                                                                                                                                                                                                                                                                 Title  \
Persepolis                                         Sleepy History                      Sun      27 Apr 2025 16:30:00 -0000 Persepolis—once a dazzling jewel of the ancient...  filled with towering columns                      intricate carvings                                   and the whispers of kings. Built to impress a...   
Explore history's most intriguing stories           people                              places  events                      and mysteries                                      delivered in a supremely calming atmosphere. I... Sleepy History is the perfect bedtime companion...    pulling your mind away from any raci

Map:   0%|          | 0/754 [00:00<?, ? examples/s]

Dataset ready! Number of examples: 754


In [ ]:
import glob
all_files = glob.glob(os.path.join(r'/content', '*.csv'))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

df.to_csv(f'new_all_podcasts.csv', index=False)

In [ ]:
import copy
just_in_case = copy.deepcopy(dataset)

# Optimizer and scheduler

In [ ]:
print(dataset[0])

{'input_ids': tensor([128000,  97012,    323,  34944,    320,   7560,    461,    340,    220,
          2366,     19]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}


In [ ]:
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-4)
num_epochs = 10
num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = "cuda" if torch.cuda.is_available() else "cpu"

# Test function

In [ ]:
def test_model(model, tokenizer, prompt="Cats in trees", max_new_tokens=500, device='cuda'):
    model.eval()

    # Make sure pad_token is set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,  # stop generation at EOS
            pad_token_id=tokenizer.pad_token_id,  # treat PAD safely
            do_sample=True,        # optional, adds randomness for more “natural” output
            top_p=0.9,             # nucleus sampling
            temperature=0.7        # controls randomness
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [ ]:
result = test_model(model, tokenizer, prompt="Boring information about puppies")
print(result)

Boring information about puppies
What a lot of people don’t know about puppies is that they are born without much of the skills and knowledge that we take for granted in adult dogs. In fact, the only thing they can do is eat and sleep. They have no idea how to open their eyes, or walk, or run, or play, or even bark. It’s amazing that they can even survive on their own.
I have seen some puppies that are so small that they can’t even stand on their own. They are so small that they can’t even see. They have to rely on their mother’s instincts to guide them. They can’t even see their mother. They are so small that they have to rely on their mother’s instincts to guide them.
This is why it is so important to be careful about what you feed your puppy. You should be careful about what you feed your puppy, because you dont want your puppy to grow up to be a big, fat, lazy, stupid puppy. You want your puppy to be able to stand on its own and walk around.
We all know that puppies are not born wi

In [ ]:
result = test_model(model, tokenizer, prompt="I will tell you about puppies")
print(result)

I will tell you about puppies who are born and raised in our family. We have 3 dogs and 6 puppies. The puppies are born on February 7, 2015 and we have 4 males and 2 females. The mother is a German Shepherd Dog and the father is a American Cocker Spaniel. The puppies are 4 months old and they have a great character and are very playful. They are very good with people and other dogs and they love to play. They are very good with kids and they are very friendly. They are very good with people and other dogs and they love to play. They are very good with kids and they are very friendly. They are very good with people and other dogs and they love to play. They are very good with kids and they are very friendly. They are very good with people and other dogs and they love to play. They are very good with kids and they are very friendly. They are very good with people and other dogs and they love to play. They are very good with kids and they are very friendly. They are very good with people 

In [ ]:
result = test_model(model, tokenizer, prompt="Here is a long boring story about puppies")
print(result)

Here is a long boring story about puppies. It is the story of how I found my puppy, and it is a long and boring story. But I will tell it anyway.
I was walking through a dog park in the city, and I saw a little puppy sitting on the grass, watching the sun go down. I couldn’t believe it was real. I was so excited to find my puppy. I had been searching for a puppy for a long time, and I finally found one.
It was a long and boring story, but it was a good story. It was about a puppy named Buddy who was born with a rare condition that made him sleep for hours at a time. He was so excited to find his puppy that he didn’t care that it was a long and boring story. He was so excited to find his puppy that he didn’t care that it was a long and boring story. He was so excited to find his puppy that he didn’t care that it was a long and boring story.
Buddy was born with a rare condition that made him sleep for hours at a time. He was so excited to find his puppy that he didn’t care that it was a 

In [ ]:
result = test_model(model, tokenizer, prompt="Welcome to my long boring podcast, todays episode is about puppies")
print(result)

Welcome to my long boring podcast, todays episode is about puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppies and puppi

In [ ]:
result = test_model(model, tokenizer, prompt="I will tell you about puppies")
print(result)

I will tell you about puppies and other things, but first I must tell you about my trip to the dog park.
We have been going to the dog park for a few weeks now. We go in the early morning, before the sun is up, and we go in the evening, when the sun is just going down. We go when it is cold and we go when it is warm. We go when the weather is bad and we go when the weather is good.
We go to the dog park, because I love to watch my dog play. I love to watch her run and jump and chase after other dogs. I love to watch her dig holes and bury bones. I love to watch her run and jump and chase after other dogs.
I have been going to the dog park for a few weeks now. I have been going to the dog park for a few weeks now. I have been going to the dog park for a few weeks now.
I love to watch her run and jump and chase after other dogs. I love to watch her dig holes and bury bones. I love to watch her run and jump and chase after other dogs.
The dog park is a great place to watch dogs play. It’s

In [ ]:
result = test_model(model, tokenizer, prompt="Welcome to tonight’s podcast to help you sleep, about puppies")
print(result)

Welcome to tonight’s podcast to help you sleep, about puppies. The puppy stage is the best stage of life. A puppy will bring you joy and happiness. A puppy will give you unconditional love. A puppy will be your best friend. But, a puppy is also a lot of work. And, if you’re not ready for the commitment, a puppy is not for you.
The podcast will start with a brief introduction to puppies and the benefits they can bring to your life. It will then go into the stages of puppyhood, from the cute and cuddly to the naughty and destructive. It will also discuss the importance of training and socialization, as well as the joys of watching your puppy grow up.


In [ ]:
result = test_model(model, tokenizer, prompt="Welcome to tonight’s podcast to help you sleep, about puppies")
print(result)

Welcome to tonight’s podcast to help you sleep, about puppies. I’m not talking about the cute ones that will be all over the internet this weekend, but the kind of puppies that are still in the womb. I’m talking about the puppies that have not been born yet.
Now, I know what you’re thinking. What does this have to do with sleep? Well, the answer is that we can learn a lot about sleep by studying the development of puppies in the womb. Let’s take a look at what we know so far.
First, it’s important to note that puppies are not born with the ability to sleep. In fact, they don’t even have a sleep/wake cycle until they are about three weeks old. That’s when they start to open their eyes and move around a bit. But even then, they are not very good at sleeping.
Puppies in the womb are very active, moving around a lot and practicing breathing. They also have a lot of energy, and they are very curious. All of this activity can make it difficult for them to fall asleep.
But as they get older, 

In [ ]:
def test_model(model, prompt="Cats in trees"):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=500)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

In [ ]:
test_model(model)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Cats in trees are a common sight, but when they are up high, it can be hard to reach them. If you are looking for a way to get your cat down from a tree, you can use a cat tree ladder. A cat tree ladder is a ladder that is specifically designed to help you reach your cat when they are up high. This ladder is made with a wide base so that it will not tip over, and it has steps that are spaced far apart so that your cat can easily climb up and down.\nHow To Get A Cat Down From A Tree\nThere are a few ways to get a cat down from a tree. One way is to use a ladder. Another way is to use a cat tree ladder.\n– A cat tree ladder – A cat tree – A ladder\nIf the cat is high up in the tree, you may need to use a ladder to reach it\nGet a cat tree ladder\nThe cat will likely be scared and will not want to come down\nIf the cat is on a low branch, you can try to coax it down with food or a toy\n– If the cat is on a low branch, you can try to coax it down with food or a toy. – If the cat is on a h

# LoRa fine tuning

In [ ]:
print("Starting LoRA fine-tuning...")
start_time = time.time()

for epoch in range(num_epochs):
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for step, batch in enumerate(progress_bar):
        model.train()
        # print(batch.items())
        batch = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask"]}

        outputs = model(**batch, labels=batch["input_ids"])
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        elapsed = time.time() - start_time
        completed_steps = epoch * len(train_loader) + step + 1
        total_steps = num_epochs * len(train_loader)
        eta = elapsed / completed_steps * (total_steps - completed_steps)
        progress_bar.set_postfix({"loss": loss.item(), "ETA (s)": int(eta)})

        # Save checkpoint every 100 steps
        if step % 300 == 0:
            checkpoint_path = os.path.join(save_dir, f"checkpoint_step_{completed_steps}.pt")
            torch.save(model.state_dict(), checkpoint_path)
            checkpoint_path = os.path.join(save_dir, f"lora_adapter_step_{completed_steps}")
            model.save_pretrained(checkpoint_path)

Starting LoRA fine-tuning...


Epoch 2:   0%|          | 0/377 [00:01<?, ?it/s, loss=0.00117, ETA (s)=1756]


KeyboardInterrupt: 

In [ ]:
print(dataset[0])

{'input_ids': tensor([128000,    323,    279,  89248,    315,  45619,     13,  34154,    311,
         10098,    323,  31740,    198,    439,  71057,    505,    279,   3347,
         30373,   8641,    499,   1139,    264,  26733,    323,  58105,   6212,
            13, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 12

In [ ]:
# test
prompt = "Once upon a time"
print("Test output after training:", test_model(model))

Test output after training: Cats in trees (Encore)
nan


In [ ]:
# test
prompt = "Once upon a time"
print("Test output after training:", test_model(model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Test output after training: Cats in trees (Encore)
nan


In [ ]:
# -----------------------
# Save final LoRA model
# -----------------------
final_dir = os.path.join(save_dir, "final_lora_model")
model.save_pretrained(final_dir)
tokenizer.save_pretrained(final_dir)
print(f"Model saved to {final_dir}")

# Loading it back

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

In [ ]:
base_model2 = AutoModelForCausalLM.from_pretrained(model_id)
model2 = PeftModel.from_pretrained(base_model2, "/content/lora_checkpoints/lora_adapter_step_test")
model2.eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_fea

In [ ]:
checkpoint_path = os.path.join(save_dir, f"lora_adapter_step_test")
model.save_pretrained(checkpoint_path)